# ベルマン残差と期待二乗TD誤差

今日は[森村(2019)](https://www.amazon.co.jp/%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%83%95%E3%82%A7%E3%83%83%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-%E6%A3%AE%E6%9D%91-%E5%93%B2%E9%83%8E/dp/4065155916/ref=sr_1_3?adgrpid=81262430850&gclid=Cj0KCQjwk7ugBhDIARIsAGuvgPYqZnWcQryNTVWKDJT8whZNJ-h73ZqHT6qF2I9bXprKne2GkLHavEIaArExEALw_wcB&hvadid=649107616291&hvdev=c&hvlocphy=1009180&hvnetw=g&hvqmt=e&hvrand=10477133917986279598&hvtargid=kwd-898864963330&hydadcr=15817_13644392&jp-ad-ap=0&keywords=%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92+%E6%A3%AE%E6%9D%91&qid=1678699533&sr=8-3)のp.168を題材にベルマン残差と期待二乗誤差について解説していきます．

---

まず，強化学習は状態価値関数 $V(s)$ や状態価値関数 $Q(s,a)$ などを使って学習を進めていくのですが，状態の空間が多い時などは次元数が大きく，例えば，テーブル形式で学習を進めると計算量が大きくなってしまいますね．

そこで，価値関数を何らかの特徴量に基づいた関数と重みで表現してあげようというのが価値関数の関数近似と言われているものです．

代表例はニューラルネットワークですね．

しかしながら今回の例では，非線形関数(ニューラルネットワークなど)ではなく，線形関数を題材に話を進めていきます．

---

下に関数近似を行った状態価値関数を示します．

$$
\hat V_{\omega}(s) \triangleq \omega ^\top \phi(s)   

$$


ここで $\phi: S \rightarrow \mathbb{R}^d$ は基底関数や特徴ベクトル，$\omega$ はd次元の重みとします．

この $\phi$ を行列の形式に直すと, $(S\times d)$ となることに気をつけてください．



関数近似をする際は関数近似器の集合を考えます．

強化学習ではベルマン作用素 $\textbf{B}$ を適用して価値関数を更新していくのですが，関数近似で行うとベルマン作用素を適用した $\textbf{BV}$ が関数近似器の集合に入っていない場合があることが注意です．

対処法は直交射影作用素を $\textbf{BV}$ に適用して，関数近似器の集合内に写すことです．

ここからの説明ではベルマン作用素を適用しても関数近似器の集合には入ると仮定して進めていきます．


---

ここからは線形関数近似器に対する損失関数を導入していきます．

まずベルマン誤差 $L_{\mathrm{BR}}$ は次のようになります．

$\mu(s)$ は初期分布などが適用できます．

$$
L_{\mathrm{BR}}(\boldsymbol{\omega}) \triangleq \sum_{s \in \mathcal{S}} \mu(s)\left(\hat{V}_{\boldsymbol{\omega}}(s)-B \hat{V}_{\boldsymbol{\omega}}(s)\right)^2
$$



次に上の損失の最小化について考えます，

先程の線形関数近似器$\hat V_{\omega}(s) \triangleq \omega ^\top \phi(s)$ を使って書き換えると，

$$
\begin{aligned}
L_{\mathrm{BR}}(\boldsymbol{\omega}) & =\sum_{s \in \mathcal{S}} \mu(s)\left(V_{\boldsymbol{\omega}}(s)-\mathbb{E}^\pi\left[g\left(s, A_t\right)+\gamma \hat{V}_{\boldsymbol{\omega}}\left(S_{t+1}\right) \mid S_t=s\right]\right)^2 \\
& =\sum \mu(s)\left\{\boldsymbol{\omega}^{\top}\left(\phi(s)-\gamma \bar{\phi}_{+1}^\pi(s)\right)-\bar{g}^\pi(s)\right\}^2
\end{aligned}
$$

ここで $\bar{g}^\pi(s) \triangleq \mathbb{E}^{\pi}[g(s,A_t)|S_t=s]$, $\bar{\phi}_{+1}^\pi(s) \triangleq \mathbb{E}^{\pi}[\phi(S_{t+1})|S_t=s]$ とします．

この上で書き換えた式においての重み $\omega$ について偏微分した式について考えます．

$$
\frac{\partial L_{\mathrm{BR}}}{\partial \omega} = 0
$$

を $\omega$ についての等式に書き換えると，
$$
\begin{aligned}
\omega=\left\{\sum_{s \in \mathcal{S}} \mu(s)(\phi(s)-\right. & \left.\left.\gamma \bar{\phi}_{+1}^\pi(s)\right)\left(\phi(s)-\gamma \bar{\phi}_{+1}^\pi(s)\right)^{\top}\right\}^{-1} \\
& \times \sum_{s \in \mathcal{S}} \mu(s) \bar{g}^\pi(s)\left(\phi(s)-\gamma \bar{\phi}_{+1}^\pi(s)\right)
\end{aligned}
$$

になり上の式に基づいて重みを計算していけばいいことがわかります．

ここでポイントとなるのは第1項目の $\bar{\phi}_{+1}^\pi(s)$ の部分が2つあるのですが，それぞれ違う状態についてサンプリングしなければいけないことが知られています．

状態遷移が決定的である時の場合だけこれは考慮しなくてもいいとされています．

これを考慮しないで強化学習の問題を解く方法は有名なTD誤差というものですね．

---

コードの説明をしていきます．

環境はp.168を基にします．



今回は損失関数の重み $\mu(s)$ を1にして考えていきたいと思います．

In [71]:
import jax 

import jax.numpy as jnp
import numpy as np
from typing import NamedTuple,Optional
import matplotlib.pyplot as plt

S = 2 #state_size
A = 2  #action_size
S_array = np.arange(S) #state_set
A_array = np.arange(A) #action_set
gamma = 0.5

rew = np.array([0,6]).reshape(2,1) #報酬

P = np.array([[0.9,0.1],[0.1,0.9]]) #遷移行列

# MDPの定義

class MDP(NamedTuple):
    S_array : np.array
    A_array : np.array
    gamma : float
    rew : np.array
    P : np.array
    phi : np.array
    horizon : int
    

    @property
    def S(self) -> int:
        return len(self.S_array)

    @property
    def A(self) -> int:
        return len(self.A_array)
d = 2
phi = np.array([[1,0],[0,1]])
horizon = int(1 / (1 - gamma))
mdp = MDP(S_array,A_array,gamma,rew,P,phi,horizon)
print(mdp.rew.shape)
print(mdp.P.shape)


(2, 1)
(2, 2)


In [75]:

@jax.jit
def get_greedy(mdp: MDP, w: jnp.ndarray) -> jnp.ndarray:
    S, A = mdp.S, mdp.A
    # 行列形式の演算も最適作用素で行いたいので、1番価値が高い, 行動の列だけを各状態で1にする。
    pi_max = jnp.zeros((S, A))
    pi_max = pi_max.at[jnp.arange(S), (mdp.rew + mdp.gamma * (w @ mdp.phi.T)).argmax(axis=1)].set(1)
    return pi_max

#L_BRの実装
@jax.jit
def bellman_residual_error(mdp: MDP, w: jnp.ndarray, pi: jnp.ndarray) -> jnp.ndarray:
    return (w @ (mdp.phi - mdp.gamma * (pi * mdp.phi).sum(axis=-1)) - (pi * mdp.rew).sum(axis=-1))**2


@jax.jit
def caluculation(mdp: MDP) -> jnp.ndarray:
    
    def backup(w):

        pi = get_greedy(mdp, w)
        A = (mdp.phi - mdp.gamma * jnp.multiply(pi, mdp.phi).sum(axis=-1)).T @ (mdp.phi - mdp.gamma * jnp.multiply(pi, mdp.phi).sum(axis=-1))
        b = jnp.multiply(pi, mdp.rew).sum(axis=-1) @ (mdp.phi - mdp.gamma * jnp.multiply(pi, mdp.phi).sum(axis=-1)).T
        w = jnp.linalg.solve(A, b)
        return w
    
    initial_w = jax.random.uniform(jax.random.PRNGKey(0), (mdp.phi.shape[-1],))
    body_fn = lambda i, initial_w: backup(initial_w)
    return jax.lax.fori_loop(0, mdp.horizon+1000, body_fn, initial_w)


In [76]:
final_w = caluculation(mdp) #最終状態の重み
final_pi = get_greedy(mdp,final_w) 
error = bellman_residual_error(mdp,final_w,final_pi).max() #最終状態の重みを使った損失

In [84]:
print(f'最終的な損失は{error}')

最終的な損失は9.0
